In [29]:
import librosa
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
import torch.nn.functional as F
from torchvision.transforms import Compose
import random
from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Setup

## Setting up variables according to audio source

Important note: Space and Enter keys are denoted as the following by the model:
- Space -> -
- Enter -> +

In [30]:
# Select 0 to use files from Dataset-for-Binary, 1 for custom dataset with normalized audio, 2 for custom dataset with normalized and denoised audio
dataset_choice=2

In [31]:
if dataset_choice == 0: 
    #Dataset-for-Binary
    MBP_AUDIO_DIR = 'Dataset-for-Binary/base-audio/'
    keys_s = '1234567890QWERTYUIOPASDFGHJKLZXCVBNM'
    labels = list(keys_s)
    keys = ['audio_' + k + '.wav' for k in labels]

    # Working parameters for the dataset
    # Dataset-for-Binary: n_fft = 1024, hop_length = 225, before = 2400, after = 12000
    n_fft = 1024 
    hop_length = 225 
    before = 2400 
    after = 12000 

    audio_descr="Using audio from Dataset-for-Binary."

elif dataset_choice == 1: 
    #Custom dataset with normalized audio
    MBP_AUDIO_DIR = 'Dataset-custom-audio/base-audio-normalized-only/' #for custom audio
    keys_s = '1234567890ABCDEFGHIJKLMNÑOPQRSTUVWXYZ+-' #for custom audio
    labels = list(keys_s)
    keys = [ k + '.wav' for k in labels] #for custom audio

    # Normalized only audio: n_fft = 7, hop_length = 4450, before = 2400, after = 12000
    n_fft = 7 
    hop_length = 4450 
    before = 2400 
    after = 12000 

    audio_descr="Using custom audio with normalized audio."

elif dataset_choice == 2: 
    #Custom dataset with normalized and denoised audio
    MBP_AUDIO_DIR = 'Dataset-custom-audio/base-audio-denoised-normalized/' #for custom audio
    keys_s = '1234567890ABCDEFGHIJKLMNÑOPQRSTUVWXYZ+-' #for custom audio
    labels = list(keys_s)
    keys = [ k + '.wav' for k in labels] #for custom audio

    # Normalized and denoised audio: n_fft = 9, hop_length = 500, before = 2400, after = 12000
    n_fft = 10
    hop_length = 10
    before = 2400 
    after = 12000 

    audio_descr="Using custom audio with normalized and denoised audio."

else: 
    #defaults to 0
    print("Invalid dataset choice, defaulting to 0")
    MBP_AUDIO_DIR = '../Dataset-for-Binary/base-audio/'
    keys_s = '1234567890QWERTYUIOPASDFGHJKLZXCVBNM'
    labels = list(keys_s)
    keys = ['audio_' + k + '.wav' for k in labels]

    # Working parameters for the dataset
    # Dataset-for-Binary: n_fft = 1024, hop_length = 225, before = 2400, after = 12000
    n_fft = 1024 
    hop_length = 225 
    before = 2400 
    after = 12000 

    audio_descr="Using audio from Dataset-for-Binary."

data_dict = {'Key':[], 'File':[]}
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
num_classes=len(labels)

## Defining basics

In [32]:
# waveform function for me to not bang my keyboard
def disp_waveform(signal, sr=None, color='blue'):
    plt.figure(figsize=(7,2))
    return librosa.display.waveshow(signal, sr=sr, color=color)

In [33]:
def isolator(signal, sample_rate, n_fft, hop_length, before, after, threshold, show=False):
    strokes = []
    # -- signal'
    if show:
        disp_waveform(signal, sr=sample_rate)
    fft = librosa.stft(signal, n_fft=n_fft, hop_length=hop_length)
    energy = np.abs(np.sum(fft, axis=0)).astype(float)
    # norm = np.linalg.norm(energy)
    # energy = energy/norm
    # -- energy'
    if show:
        disp_waveform(energy)
    threshed = energy > threshold
    # -- peaks'
    if show:
        disp_waveform(threshed.astype(float))
    peaks = np.where(threshed == True)[0]
    peak_count = len(peaks)
    prev_end = sample_rate*0.1*(-1)
    # '-- isolating keystrokes'
    for i in range(peak_count):
        this_peak = peaks[i]
        timestamp = (this_peak*hop_length) + n_fft//2
        if timestamp > prev_end + (0.1*sample_rate):
            keystroke = signal[timestamp-before:timestamp+after]
            # strokes.append(torch.tensor(keystroke)[None, :])
            # keystroke = transform(keystroke)
            strokes.append(keystroke)
            if show:
                disp_waveform(keystroke, sr=sample_rate)
            prev_end = timestamp+after
    return strokes

In [34]:
def create_dataset(n_fft, hop_length, before, after, keys, custom_audio=False):
    for i, File in enumerate(keys):
        loc = MBP_AUDIO_DIR + File
        samples, sr = librosa.load(loc)
        prom = 0.06
        step = 0.005
        strokes = isolator(samples, sr, n_fft, hop_length, before, after, prom, False )
        print(f'File {File} length: {len(strokes)}')
        label = [labels[i]]*len(strokes)
        #works fine here
        data_dict['Key'] += label
        data_dict['File'] += strokes

    df = pd.DataFrame(data_dict)
    mapper = {}
    counter = 0
    for l in df['Key']:
        if not l in mapper:
            mapper[l] = counter
            counter += 1
        print(mapper)
    df.replace({'Key': mapper}, inplace = True)

    return df

## Creating dataset from chosen audio files

In [35]:
data_dict = {'Key':[], 'File':[]} #Resets data_dict for testing purposes
mbp_dataset = create_dataset(n_fft, hop_length, before, after, keys)
mbp_dataset

File 1.wav length: 44
File 2.wav length: 42
File 3.wav length: 44
File 4.wav length: 44
File 5.wav length: 46
File 6.wav length: 45
File 7.wav length: 44
File 8.wav length: 42
File 9.wav length: 41
File 0.wav length: 46
File A.wav length: 58
File B.wav length: 57
File C.wav length: 56
File D.wav length: 54
File E.wav length: 58
File F.wav length: 58
File G.wav length: 56
File H.wav length: 57
File I.wav length: 59
File J.wav length: 56
File K.wav length: 49
File L.wav length: 49
File M.wav length: 44
File N.wav length: 43
File Ñ.wav length: 41
File O.wav length: 46
File P.wav length: 45
File Q.wav length: 49
File R.wav length: 47
File S.wav length: 45
File T.wav length: 47
File U.wav length: 43
File V.wav length: 40
File W.wav length: 42
File X.wav length: 43
File Y.wav length: 42
File Z.wav length: 43
File +.wav length: 56
File -.wav length: 45
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0}
{'1': 0

,Key,File
0,0,"[-1.2529549e-06, -2.2429464e-05, -5.6740428e-0..."
1,0,"[-0.0013983386, -0.0047972584, -0.002139621, -..."
2,0,"[0.010180487, 0.011449853, 0.012868414, 0.0094..."
3,0,"[-0.0064186836, -0.007577601, -0.008315172, -0..."
4,0,"[-0.00059519673, -0.0007372622, 0.0011575967, ..."
...,...,...
1861,38,"[-0.0055503063, -0.005259658, -0.0045290384, -..."
1862,38,"[0.007921062, 0.008146462, 0.008569973, 0.0093..."
1863,38,"[-0.00076276565, -0.007117328, -0.008020098, -..."
1864,38,"[-0.01468038, -0.015496649, -0.016122773, -0.0..."


In [36]:
audio_samples = mbp_dataset['File'].values.tolist()
labels = mbp_dataset['Key'].values.tolist()

audioDataset = np.array(audio_samples, dtype = object)
print(audio_samples[0].shape)
mfcc = librosa.feature.mfcc(y=audio_samples[0], sr=44100) # shape: (n_mfcc, t)
print(mfcc.shape)
# labels = np.array(labels)

(14400,)
(20, 29)


In [37]:
audio_samples[0]


array([-1.2529549e-06, -2.2429464e-05, -5.6740428e-06, ...,
        4.3018744e-04, -2.8467514e-03, -2.0093345e-03], dtype=float32)

## Defining model and specifying dataset


In [38]:
class TimeShifting():
    def __call__(self, samples):
#       samples_shape = samples.shape
        samples = samples.flatten()
        
        shift = int(len(samples) * 0.4) #Max shift (0.4)
        random_shift = random.randint(0, shift) #Random number between 0 and 0.4*len(samples)
        data_roll = np.roll(samples, random_shift)
        return data_roll

In [39]:
def time_shift(samples):
    samples = samples.flatten()
    shift = int(len(samples) * 0.4) #Max shift (0.4)
    random_shift = random.randint(0, shift) #Random number between 0 and 0.4*len(samples)
    data_roll = np.roll(samples, random_shift)
    return data_roll

In [40]:
from skimage.transform import resize


class ToMelSpectrogram:
    def __init__(self, audio_length=14400):
        self.audio_length = audio_length

    def __call__(self, samples):
        if len(samples) > self.audio_length:
            samples = samples[:self.audio_length]
        elif len(samples) < self.audio_length:
            samples = np.pad(samples, (0, self.audio_length - len(samples)), mode='constant')

        mel_spec = librosa.feature.melspectrogram(y=samples, sr=44100, n_mels=64, n_fft=1024, hop_length=225)
        mel_spec_resized = resize(mel_spec, (64, 64), anti_aliasing=True)
        mel_spec_resized = np.expand_dims(mel_spec_resized, axis=0)
        return torch.tensor(mel_spec_resized)


class ToMelSpectrogramMfcc:
    def __init__(self, audio_length=14400):
        self.audio_length = audio_length

    def __call__(self, samples):
        if len(samples) > self.audio_length:
            samples = samples[:self.audio_length]
        elif len(samples) < self.audio_length:
            samples = np.pad(samples, (0, self.audio_length - len(samples)), mode='constant')

        mel_spec = librosa.feature.melspectrogram(y=samples, sr=44100, n_mels=64, n_fft=n_fft, hop_length=hop_length)
        mel_spec = librosa.feature.mfcc(S=librosa.power_to_db(mel_spec))
        mel_spec_resized = resize(mel_spec, (64, 64), anti_aliasing=True)
        mel_spec_resized = np.expand_dims(mel_spec_resized, axis=0)

        return torch.tensor(mel_spec_resized)


class ToMfcc:
    def __init__(self, audio_length=14400):
        self.audio_length = audio_length

    def __call__(self, samples):
        if len(samples) > self.audio_length:
            samples = samples[:self.audio_length]
        elif len(samples) < self.audio_length:
            samples = np.pad(samples, (0, self.audio_length - len(samples)), mode='constant')
        
        mfcc_spec = librosa.feature.mfcc(y=samples, sr=44100)
        mfcc_spec = np.transpose(mfcc_spec)
        return torch.tensor(mfcc_spec)


In [41]:
transform = Compose([ToMelSpectrogram()])
transform_mfcc = Compose([ToMfcc()])

In [42]:
audio_samples_new = audio_samples.copy() # audio samples CNN

for i, sample in enumerate(audio_samples):
    audio_samples_new.append(time_shift(sample))
    labels.append(labels[i])
    
# convert labels to a numpy array
labels = np.array(labels)
print(len(audio_samples_new))
print(len(labels))

3732
3732


In [43]:
audioDatasetFin, audioDatasetMfcc = [], []

for i in range(len(audio_samples_new)):
    transformed_sample = transform(audio_samples_new[i])
    transformed_mfcc = transform_mfcc(audio_samples_new[i])
    audioDatasetFin.append((transformed_sample, labels[i]))
    audioDatasetMfcc.append((transformed_sample, transformed_mfcc, labels[i]))

In [44]:
len(audioDatasetFin)

3732

In [45]:
audioDatasetMfcc[0][0].shape

torch.Size([1, 64, 64])

In [46]:
import time

class MfccLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_classes,dropout=0.2, ):
        super(MfccLSTM, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, 3, 1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.LazyLinear(512),
            nn.ReLU(),
            nn.Linear(512, num_classes)
        )
        
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc1 = nn.LazyLinear(64)
        self.fc2 = nn.Linear(64, 16)
    
        self.fc3 = nn.LazyLinear(128)
        self.final_lstm = nn.LSTM(1, 64, batch_first=True)
        
        self.fc = nn.LazyLinear(num_classes)
    
    def forward(self, image_input, sequence_input):
        # must return shape (batch_size, num_classes) 
        # batch_size: right now is 16
        # num_classes: right now is 36
        x1 = self.conv(image_input)
        out1, _ = self.lstm(sequence_input)
        out1_dp = self.dropout(out1)
        # print(f'output of first lstm: {out1_dp.shape[1:]}')
        out2, _ = self.lstm2(out1_dp[:, -1, :])
        out2_dp = self.dropout(out2)
        # print(f'output of second lstm: {out2_dp.shape[1:]}')
        x2 = self.fc2(self.fc1(out2_dp))
        x3 = torch.cat((x1, x2), 1)
        # print(f'output of concatenation: {x3.shape[1:]}')
        # x = self.fc(final_out[:, -1, :])
        x = self.fc(x3)
        return x
    

In [47]:
# Model architecture
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.LazyLinear(512)
        self.fc2 = nn.Linear(512, num_classes)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [48]:
import time

def train_with_cross_validation(dataset, num_epochs, model_name, num_classes, patience=15, random_state=42, n_splits=10):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(dataset)):
        print(f'Fold {fold+1}/{n_splits}')
        
        # Split the dataset into training and validation sets
        train_set = Subset(dataset, train_idx)
        val_set = Subset(dataset, val_idx)
        train_loader = DataLoader(train_set, batch_size=16, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=16, shuffle=True)
        
        # Initialize model, optimizer, and loss function
        model = MfccLSTM(input_size=20, hidden_size=32, num_classes=num_classes, output_size=64)
        model = model.to(device)
        optimizer = optim.Adam(model.parameters(), lr=5e-4)
        criterion = nn.CrossEntropyLoss()
        
        best_val_acc, epochs_no_imp = 0, 0
        train_accuracies, val_accuracies = [], []

        for epoch in range(num_epochs):
            model.train()
            epoch_train_loss = 0.0
            correct_train = 0
            total_train = 0
            tic = time.perf_counter()
            
            for images, sequences, labels in train_loader:
                images = images.to(device)
                sequences = sequences.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()

                #converting labels to Long to avoid error "not implemented for Int"
                labels = labels.long()
                
                # Check that labels are within the valid range
                assert labels.min() >= 0 and labels.max() < num_classes, "Labels are out of bounds"
                
                # Forward pass
                outputs = model(images, sequences)
                loss = criterion(outputs, labels)
                epoch_train_loss += loss.item() * images.size(0)

                _, predicted_train = torch.max(outputs.data, 1)
                total_train += labels.size(0)
                correct_train += (predicted_train == labels).sum().item()
                
                # Backward pass
                loss.backward()
                optimizer.step()
            
            toc = time.perf_counter()
            time_taken = toc - tic
            
            epoch_train_loss /= len(train_loader.dataset)
            train_accuracy = correct_train / total_train
            train_accuracies.append(train_accuracy)
            
            # Evaluation of the model
            model.eval()
            total, correct = 0, 0
            for images, sequences, labels in val_loader:
                images = images.to(device)
                sequences = sequences.to(device)
                labels = labels.to(device)

                outputs = model(images, sequences)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            
            val_accuracy = correct / total
            val_accuracies.append(val_accuracy)
            if (epoch + 1) % 5 == 0:
                print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {epoch_train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Accuracy: {val_accuracy:.4f}, Iter Time: {time_taken:.2f}s")
                
            if val_accuracy > best_val_acc:
                best_val_acc = val_accuracy
                epochs_no_imp = 0
                best_model_state = model.state_dict()  # Save the best model
            else:
                epochs_no_imp += 1
            if epochs_no_imp >= patience:
                print(f'Early stopping after {epoch+1} epochs')
                model.load_state_dict(best_model_state)  # Load the best model
                break
        
        fold_results.append((epoch+1, best_val_acc))
        print(f'Fold {fold+1} Best Validation Accuracy: {best_val_acc:.4f}')
    torch.save(model.state_dict(), model_name)

    return fold_results

In [49]:
def predict_mfcc(dataset, model_path, device_external, keys, num_classes):
    images_test_set = [t[0] for t in dataset]
    sequences_test_set = [t[1] for t in dataset]
    
    images = torch.stack(images_test_set)
    sequences = torch.stack(sequences_test_set)
    device = torch.device(device_external) #default to mps
    images = images.to(device)
    sequences = sequences.to(device)
    model = MfccLSTM(input_size=20, hidden_size=32, num_classes=num_classes, output_size=64)
    model = model.to(device)
    model.load_state_dict(torch.load(model_path,map_location=device))
    model.eval()
    
    with torch.no_grad():
        outputs = model(images, sequences)
        _, predicted = torch.max(outputs.data, 1)

    pred = []
    phrase = predicted.tolist()
    for i in range(len(phrase)):
        pred.append(keys[phrase[i]])

    pred_df = pd.DataFrame(pred)
    return pred_df

In [50]:
def save_csv(model_name, num_epochs, description, accuracy, precision, recall, f1_score):
    csv_file_path = 'model_comparison.csv'
    
    # Read the existing CSV file into a DataFrame
    try:
        df = pd.read_csv(csv_file_path)
    except FileNotFoundError:
        # If the file does not exist, create an empty DataFrame with the correct columns
        df = pd.DataFrame(columns=['Datetime', 'Name', 'Epochs', 'Description', 'Accuracy', 'Precision', 'Recall', 'F1'])
        
    # Data to append
    current_datetime = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    # Remove newline characters from the description
    description = description.replace('\n', ' ').replace('\r', ' ')
    
    # Create a new column with the relevant information
    new_data = {
        'Datetime': [current_datetime],
        'Name': [model_name],
        'Epochs': [num_epochs],
        'Description': [description],
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1': [f1_score],
    }
    
    new_df = pd.DataFrame(new_data)
    
    df = pd.concat([df, new_df], ignore_index=True)
    
    # Save the updated DataFrame back to the CSV file
    df.to_csv(csv_file_path, index=False)

# Running with audio from Dataset-for-Binary

In [51]:
# current random state to split the dataset
random_state = 42

# values for current run
train_final_set, test_set = train_test_split(audioDatasetMfcc, test_size=0.2, random_state=random_state)
num_epochs = 100
main_architecture = "CNN_LSTM"
currday = datetime.today().strftime('%Y-%m-%d')
model_name = f"model_multiclass_{num_epochs}_{main_architecture}_{currday}.pth"
description = "2 layer CNN (32 and 64 output channels) with final 2 Dense Layers (512 and "+str(num_classes)+") result concatenated with \n 2 LSTMs (hidden_size=32),  from mfcc with 2 Dense Layers (64 and 16) with a final Lazy Linear layer output of "+str(num_classes)+". "+audio_descr+" n_fft="+str(n_fft)+", hop_length="+str(hop_length)


# Training part
fold_stats = train_with_cross_validation(train_final_set, num_epochs, model_name, num_classes,random_state=random_state)
max_val = 0
real_num_epochs = 0
for fold_stat in fold_stats: #using folds instead of LOO
    if fold_stat[1] > max_val:
        max_val = fold_stat[1]
        real_num_epochs = fold_stat[0]


Fold 1/10


c:\Users\Kat\.conda\envs\tesis\lib\site-packages\torch\nn\modules\lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Epoch [5/100], Train Loss: 1.4907, Train Accuracy: 0.5097, Val Accuracy: 0.4582, Iter Time: 2.13s
Epoch [10/100], Train Loss: 0.9082, Train Accuracy: 0.6969, Val Accuracy: 0.5686, Iter Time: 1.54s
Epoch [15/100], Train Loss: 0.7079, Train Accuracy: 0.7628, Val Accuracy: 0.6154, Iter Time: 1.84s
Epoch [20/100], Train Loss: 0.5393, Train Accuracy: 0.8101, Val Accuracy: 0.6087, Iter Time: 1.79s
Epoch [25/100], Train Loss: 0.4643, Train Accuracy: 0.8555, Val Accuracy: 0.6355, Iter Time: 1.61s
Epoch [30/100], Train Loss: 0.4186, Train Accuracy: 0.8745, Val Accuracy: 0.6488, Iter Time: 1.71s
Epoch [35/100], Train Loss: 0.2169, Train Accuracy: 0.9255, Val Accuracy: 0.6421, Iter Time: 1.75s
Early stopping after 37 epochs
Fold 1 Best Validation Accuracy: 0.6555
Fold 2/10
Epoch [5/100], Train Loss: 1.6461, Train Accuracy: 0.4758, Val Accuracy: 0.3478, Iter Time: 1.76s
Epoch [10/100], Train Loss: 1.0314, Train Accuracy: 0.6649, Val Accuracy: 0.4615, Iter Time: 1.88s
Epoch [15/100], Train Loss: 0.

In [52]:
# Prediction part
prediction = predict_mfcc(test_set, model_name, device, keys_s,num_classes)
labels_set = [t[2] for t in test_set]
final_labels_set = [keys_s[ind] for ind in labels_set]

# Metrics calculation
accuracy = accuracy_score(final_labels_set, prediction[0])
precision = precision_score(final_labels_set, prediction[0], average='macro')
recall = recall_score(final_labels_set, prediction[0], average='macro')
f1 = sklearn.metrics.f1_score(final_labels_set, prediction[0], average='macro')

# Save in csv file
save_csv(model_name, real_num_epochs, description, accuracy, precision, recall, f1)

# Print results
print("Final Results!")
print(f"Model: {model_name}")
print(description)
print(f"Epochs: {real_num_epochs}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Final Results!
Model: model_multiclass_100_CNN_LSTM_2024-09-02.pth
2 layer CNN (32 and 64 output channels) with final 2 Dense Layers (512 and 39) result concatenated with 
 2 LSTMs (hidden_size=32),  from mfcc with 2 Dense Layers (64 and 16) with a final Lazy Linear layer output of 39. Using custom audio with normalized and denoised audio. n_fft=10, hop_length=10
Epochs: 48
Accuracy: 0.6091030789825971
Precision: 0.6386252282685957
Recall: 0.6266708350219435
F1 Score: 0.6174939274466684


In [53]:
import csv

def empty_file(csv_file_path):
    # Read the header (first row) of the CSV file
    with open(csv_file_path, 'r') as file:
        reader = csv.reader(file)
        header = next(reader)  # Read the first row (header)
    
    # Write only the header back to the CSV file
    with open(csv_file_path, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(header)  # Wr`ite the header back to the file


In [54]:
# empty_file('model_comparison.csv')